In [1]:
# imports

import os
import os.path
import cv2
import glob
import h5py
import tqdm
import argparse
import logging
from PIL import Image 

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_theme()
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms

import sys
sys.path.append('../')
import data, utils, models

In [2]:
# necessary variable definitions

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset = "GoPro"

# load the desired video to denoise
# video = "snowboard"
# video = "hypersmooth"
video = "rafting"
# video = "motorbike"

patch_size = 128
stride = 64
is_image = False
n_frames = 5
cpf = 3
mid = n_frames // 2
Fast = False
sigma_known = True

aug = 0

dist = 'G'
mode = 'S'
noise_std = 30
min_noise = 0
max_noise = 100

batch_size = 8
lr = 1e-4
epochs = 100

In [3]:
# UDVD

PATH = "../pretrained/blind_video_net.pt"
model, _, _ = utils.load_model(PATH, parallel=True)
model.to(device).eval()

BlindVideoNet(
  (rotate): rotate()
  (denoiser_1): Blind_UNet(
    (enc1): ENC_Conv(
      (conv1): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(9, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (conv2): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (conv3): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
  

In [4]:
# data loader

PATH = "../datasets/Set8"
test_loader = data.build_dataset("Set8", PATH, batch_size=1, n_frames=n_frames)

In [5]:
# evaluating UDVD on Set8 dataset for multiple gaussian noise sigma values

noise_stds = [20, 30, 40, 50, 60, 70, 80]

final_psnrs = np.zeros((len(noise_stds), len(test_loader.dataset)))
if not Fast:
    mean_psnrs = np.zeros((len(noise_stds), len(test_loader.dataset)))
noisy_psnrs = np.zeros((len(noise_stds), len(test_loader.dataset)))

final_ssims = np.zeros((len(noise_stds), len(test_loader.dataset)))
if not Fast:
    mean_ssims = np.zeros((len(noise_stds), len(test_loader.dataset)))
noisy_ssims = np.zeros((len(noise_stds), len(test_loader.dataset)))

for j, noise_std in enumerate(noise_stds):
    print(f"Evaluating for sigma = {noise_std}")
    
    valid_meters = {name: utils.AverageMeter() for name in (["valid_psnr", "valid_ssim"])}
    for meter in valid_meters.values():
        meter.reset()
    
    with torch.no_grad():
        test_bar = utils.ProgressBar(test_loader)
        for i, sample in enumerate(test_bar):
            sample = sample.to(device)

            clean_image = sample[:, (mid*cpf):((mid+1)*cpf), :, :]
            noise = utils.get_noise(sample, dist = "G", mode = 'S', noise_std = noise_std)
            noisy_inputs = noise + sample
            noisy_frame = noisy_inputs[:, (mid*cpf):((mid+1)*cpf), :, :]

            N, C, H, W = sample.shape
            noise_map = (noise_std/255)*torch.ones(N, 1, H, W).to(device)
            if Fast:
                output = model(noisy_inputs, noise_map)
            else:
                out, sigma = model(noisy_inputs)
                if not sigma_known:
                    output, mean_image = utils.post_process(out, noisy_frame, model = "blind-video-net-4", sigma = sigma, device=device)
                else:
                    output, mean_image = utils.post_process(out, noisy_frame, model = "blind-video-net-4", sigma = noise_std/255., device=device)

            final_psnrs[j][i] = utils.psnr(clean_image, output, normalized=True, raw=False)
            final_ssims[j][i] = utils.ssim(clean_image, output, normalized=True, raw=False)
            valid_meters["valid_psnr"].update(final_psnrs[j][i].item())
            valid_meters["valid_ssim"].update(final_ssims[j][i].item())
            if not Fast:
                mean_psnrs[j][i] = utils.psnr(clean_image, mean_image, normalized=True, raw=False)
                mean_ssims[j][i] = utils.ssim(clean_image, mean_image, normalized=True, raw=False)

            if is_image:
                noisy_psnrs[j][i] = utils.psnr(clean_image, noisy_inputs, normalized=True, raw=False)
                noisy_ssims[j][i] = utils.ssim(clean_image, noisy_inputs, normalized=True, raw=False)
            else:
                noisy_psnrs[j][i] = utils.psnr(clean_image, noisy_frame, normalized=True, raw=False)
                noisy_ssims[j][i] = utils.ssim(clean_image, noisy_frame, normalized=True, raw=False)
            
            test_bar.log(dict(**valid_meters), verbose=True)
        
        print("- "+"Final PSNR : "+f"{np.mean(final_psnrs, axis=1)[j]}")
        if not Fast:
            print("- "+"Mean PSNR : "+f"{np.mean(mean_psnrs, axis=1)[j]}")
        print("- "+"Noisy PSNR : "+f"{np.mean(noisy_psnrs, axis=1)[j]}")

  0%|          | 0/497 [00:00<?, ?it/s]

Evaluating for sigma = 20


  0%|          | 0/497 [00:00<?, ?it/s]                                                                

- Final PSNR : 33.36094247763582
- Mean PSNR : 31.85494535638163
- Noisy PSNR : 22.467140271837014
Evaluating for sigma = 30


  0%|          | 0/497 [00:00<?, ?it/s]                                                                

...

  0%|          | 0/497 [00:00<?, ?it/s]                                                                

- Final PSNR : 29.133120757467374
- Mean PSNR : 28.60644263658853
- Noisy PSNR : 13.900299654857054
Evaluating for sigma = 70


  0%|          | 0/497 [00:00<?, ?it/s]                                                                

- Final PSNR : 28.492410872229986
- Mean PSNR : 28.047073726524
- Noisy PSNR : 12.808501911474593
Evaluating for sigma = 80


- Final PSNR : 27.942725892553494
- Mean PSNR : 27.559092612900216
- Noisy PSNR : 11.903290855998977


In [6]:
set8_data = {"sigma": noise_stds,
             "PSNR": np.mean(final_psnrs, axis=1),
             "Mean PSNR": np.mean(mean_psnrs, axis=1),
             "Noise PSNR": np.mean(noisy_psnrs, axis=1),
             "PSNR stddev": np.sqrt(np.var(final_psnrs, axis=1)),
             "Mean PSNR stddev": np.sqrt(np.var(mean_psnrs, axis=1)),
             "Noise PSNR stddev": np.sqrt(np.var(noisy_psnrs, axis=1)), 
             "SSIM": np.mean(final_ssims, axis=1),
             "Mean SSIM": np.mean(mean_ssims, axis=1),
             "Noise SSIM": np.mean(noisy_ssims, axis=1),
             "SSIM stddev": np.sqrt(np.var(final_ssims, axis=1)),
             "Mean SSIM stddev": np.sqrt(np.var(mean_ssims, axis=1)),
             "Noise SSIM stddev": np.sqrt(np.var(noisy_ssims, axis=1))
            }

df = pd.DataFrame(set8_data, columns=["sigma", "PSNR", "Mean PSNR", "Noise PSNR",
                                     "PSNR stddev", "Mean PSNR stddev", "Noise PSNR stddev",
                                     "SSIM", "Mean SSIM", "Noise SSIM",
                                     "SSIM stddev", "Mean SSIM stddev", "Noise SSIM stddev"])
df.to_csv(f"../examples/set8_UDVD_results.csv", index=False)
df

,sigma,PSNR,Mean PSNR,Noise PSNR,PSNR stddev,Mean PSNR stddev,Noise PSNR stddev,SSIM,Mean SSIM,Noise SSIM,SSIM stddev,Mean SSIM stddev,Noise SSIM stddev
0,20,33.360942,31.854945,22.467140,2.093114,2.849393,0.260859,0.914355,0.894317,0.429116,0.018616,0.026094,0.092714
1,30,32.007165,30.946133,19.182023,2.238823,2.813016,0.336716,0.888608,0.870048,0.302079,0.026503,0.032401,0.086390
2,40,30.822483,30.023061,16.929511,2.268569,2.720548,0.365760,0.861527,0.844170,0.226994,0.032584,0.038280,0.074783
3,50,29.892867,29.255268,15.238341,2.277348,2.646093,0.361968,0.836376,0.820074,0.177880,0.038111,0.043479,0.063555
4,60,29.133121,28.606443,13.900300,2.271098,2.578812,0.339208,0.813169,0.797855,0.143616,0.042941,0.047954,0.053972
5,70,28.492411,28.047074,12.808502,2.259096,2.520138,0.303829,0.791914,0.777522,0.118641,0.046754,0.051545,0.046042
6,80,27.942726,27.559093,11.903291,2.243878,2.468014,0.262840,0.772502,0.758963,0.099911,0.049576,0.054229,0.039606


In [7]:
# load the desired pretrained model; for Single Video Denoisining load the corresponding video's model

# # UDVD
# PATH = "../pretrained/blind_video_net.pt"

# # SingleVideo - Set8/GoPro - snowboard
# PATH = "../pretrained/single_video_Set8_snowboard_30.pt"

# # SingleVideo - Set8/GoPro - hypersmooth
# PATH = "../pretrained/single_video_Set8_hypersmoth_30.pt"

# SingleVideo - Set8/GoPro - rafting
PATH = "../pretrained/single_video_Set8_rafting_30.pt"

# # SingleVideo - Set8/GoPro - motorbike
# PATH = "../pretrained/single_video_Set8_motorbike_30.pt"

model, optimizer, args = utils.load_model(PATH, parallel=True)
model.to(device)
print(model)

BlindVideoNet(
  (rotate): rotate()
  (denoiser_1): Blind_UNet(
    (enc1): ENC_Conv(
      (conv1): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(9, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (conv2): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (conv3): Conv(
        (shift_down): ZeroPad2d(padding=(0, 0, 1, 0), value=0.0)
        (crop): crop()
        (replicate): ReplicationPad2d((1, 1, 1, 1))
        (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (relu): LeakyReLU(negative_slope=0.1, inplace=True)
  

In [8]:
# data loader

PATH = os.path.join("../datasets/Set8", dataset)

noise_std = 30

train_loader, test_loader = data.build_dataset("SingleVideo", PATH, batch_size=batch_size, dataset=dataset, video=video, image_size=patch_size, stride=stride, n_frames=n_frames, 
                                               aug=aug, dist=dist, mode=mode, noise_std=noise_std, min_noise=min_noise, max_noise=max_noise,
                                               sample=True, heldout=False)

In [9]:
# Test

valid_meters = {name: utils.AverageMeter() for name in (["valid_psnr", "valid_ssim"])}
mean_meters = {name: utils.AverageMeter() for name in (["mean_psnr", "mean_ssim"])}

model.eval()
for meter in valid_meters.values():
    meter.reset()
for meter in mean_meters.values():
    meter.reset()

valid_bar = utils.ProgressBar(test_loader)
running_valid_psnr = 0.0
plist = []
slist = []
for sample_id, (sample, noisy_inputs) in enumerate(valid_bar):
    with torch.no_grad():
        sample = sample.to(device)
        noisy_inputs = noisy_inputs.to(device)
        
        out, est_sigma = model(noisy_inputs)
        
        noisy_frame = noisy_inputs[:, (mid*cpf):((mid+1)*cpf), :, :]
        outputs, mean_image = utils.post_process(out, noisy_frame, model="blind-video-net", sigma=noise_std/255, device=device)
        
        valid_psnr = utils.psnr(sample[:, (mid*cpf):((mid+1)*cpf), :, :], outputs, normalized=True, raw=False)
        valid_ssim = utils.ssim(sample[:, (mid*cpf):((mid+1)*cpf), :, :], outputs, normalized=True, raw=False)
        plist.append(valid_psnr)
        slist.append(valid_ssim)
        running_valid_psnr += valid_psnr
        valid_meters["valid_psnr"].update(valid_psnr.item())
        valid_meters["valid_ssim"].update(valid_ssim.item())

        mean_psnr = utils.psnr(sample[:, (mid*cpf):((mid+1)*cpf), :, :], mean_image, normalized=True, raw=False)
        mean_ssim = utils.ssim(sample[:, (mid*cpf):((mid+1)*cpf), :, :], mean_image, normalized=True, raw=False)
        mean_meters["mean_psnr"].update(mean_psnr.item())
        mean_meters["mean_ssim"].update(mean_ssim.item())
        
        valid_bar.log(dict(**valid_meters, **mean_meters, lr=optimizer.param_groups[0]["lr"]), verbose=True)
        
running_valid_psnr /= (sample_id+1)
print("EVAL: "+valid_bar.print(dict(**valid_meters, **mean_meters, lr=optimizer.param_groups[0]["lr"])))

EVAL: valid_psnr 30.796 | valid_ssim 0.861 | mean_psnr 28.834 | mean_ssim 0.815 | lr 1.0e-04
